**[Advanced SQL Home Page](https://www.kaggle.com/learn/advanced-sql)**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,ade155d8f730a13006affc42f26dbd82b0d83ea9,f3e1a9085537bee3447c67580b4d30b827f97b8a3fa638...,2013-07-12 20:30:00+00:00,2013-07-12 20:30:00+00:00,480.0,0.0,None,None,None,None,...,0.0,10.86,Credit Card,Chicago Elite Cab Corp.,None,None,None,None,None,None
1,3fe2116d5628b3a9a6547b8ce1f480afb8f3578e,d95b99518116b5f943d75828e78c02e668ec6add7d28ba...,2013-07-12 13:45:00+00:00,2013-07-12 13:45:00+00:00,0.0,0.0,None,None,None,None,...,0.0,11.05,No Charge,Chicago Elite Cab Corp.,None,None,None,None,None,None
2,0343559825f045e6341e4ab0a9a648c6fdafdd3a,6dcd43a953f0c1bdfc49fe8260b9504af2cc45eefdfc22...,2013-09-28 05:15:00+00:00,2013-09-28 05:30:00+00:00,480.0,0.0,None,None,None,None,...,0.0,11.82,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
3,a9db489cb175bbc82894dd74c109e61c4028cbaf,335e5b94c1eb05d4b83164f78b0246b7ce44188a32d029...,2013-07-12 23:15:00+00:00,2013-07-12 23:30:00+00:00,1500.0,0.0,None,None,None,None,...,0.0,21.15,Credit Card,Choice Taxi Association,None,None,None,None,None,None
4,545fa4c18ec5f6eea912e6891c4795ac0772b87c,bd54bee5660726988fc54355ab54fc7cfd5c3917d911f9...,2013-09-27 12:00:00+00:00,2013-09-27 12:00:00+00:00,NaN,0.0,None,None,None,None,...,0.0,40.25,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame.  For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) micro-course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                          SELECT DATE(trip_start_timestamp) AS trip_date,
                              COUNT(*) as num_trips
                          FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                          WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                          GROUP BY trip_date
                          ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                              ORDER BY trip_date
                              ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                          ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-01,80461.937500
1,2016-01-02,80150.647059
2,2016-01-03,79419.611111
3,2016-01-04,79810.421053
4,2016-01-05,80293.900000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                PARTITION BY pickup_community_area
                                ORDER BY trip_start_timestamp
                            ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,20.0,2017-05-01 00:45:00+00:00,2017-05-01 01:00:00+00:00,1
1,20.0,2017-05-01 02:00:00+00:00,2017-05-01 02:00:00+00:00,2
2,20.0,2017-05-01 02:00:00+00:00,2017-05-01 02:15:00+00:00,2
3,20.0,2017-05-01 03:15:00+00:00,2017-05-01 03:30:00+00:00,4
4,20.0,2017-05-01 03:45:00+00:00,2017-05-01 04:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                   PARTITION BY taxi_id 
                                   ORDER BY trip_start_timestamp
                               ),
                               MINUTE
                       ) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,020f09fafe503bfb0f58dc3e7d9f3ee60fa94c1c1473cf...,2017-05-01 16:00:00+00:00,2017-05-01 16:00:00+00:00,NaN
1,19746bb16bdf4cfc8816650883cc6b6f8a7401ceb73a61...,2017-05-01 06:00:00+00:00,2017-05-01 06:00:00+00:00,NaN
2,19746bb16bdf4cfc8816650883cc6b6f8a7401ceb73a61...,2017-05-01 06:15:00+00:00,2017-05-01 06:15:00+00:00,15.0
3,19746bb16bdf4cfc8816650883cc6b6f8a7401ceb73a61...,2017-05-01 06:30:00+00:00,2017-05-01 06:45:00+00:00,15.0
4,19746bb16bdf4cfc8816650883cc6b6f8a7401ceb73a61...,2017-05-01 06:45:00+00:00,2017-05-01 06:45:00+00:00,0.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
q_3.hint()
#q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The `TIMESTAMP_DIFF()` function takes three arguments, where the first (`trip_start_timestamp`) and the last (`MINUTE`) are provided for you.  This function provides the time difference (in minutes) of the timestamps in the first two arguments. You need only fill in the second argument, which should use the **LAG()** function to pull the timestamp corresponding to the end of the previous trip (for the same `taxi_id`).

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---
**[Advanced SQL Home Page](https://www.kaggle.com/learn/advanced-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*